### How to use SOTA, which updates frequently, in a very short time? 

#### Bring your own container - BYOC 

In [1]:
!pip install seaborn 

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
%%bash 
cd byoc/smore/
./build_and_push.sh smore-byoc

Login Succeeded

Step 1/7 : FROM ubuntu:18.04
 ---> 2eb2d388e1a2
Step 2/7 : RUN apt-get update &&     apt-get install -y --no-install-recommends         ca-certificates         cmake         build-essential         gcc         g++         git &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 928aea19ecbf
Step 3/7 : RUN git clone https://github.com/cnclabs/smore.git && cd smore && make
 ---> Using cache
 ---> 84c80480d45a
Step 4/7 : RUN cp -r /smore /app/
 ---> Using cache
 ---> 3c8ea08e06a2
Step 5/7 : ENV PATH="/app:${PATH}"
 ---> Using cache
 ---> 0d175c678e3c
Step 6/7 : WORKDIR /app
 ---> Using cache
 ---> 44b335f17ab3
Step 7/7 : COPY train /app/
 ---> Using cache
 ---> f1a1f76c3add
Successfully built f1a1f76c3add
Successfully tagged smore-byoc:latest
The push refers to repository [230755935769.dkr.ecr.us-east-1.amazonaws.com/smore-byoc]
cf796395a59d: Preparing
714816eb6d34: Preparing
e85547b9ab30: Preparing
e1ce31771c29: Preparing
8682f9a74649: Preparing
d3a6da143c91: Prepa

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [3]:
%env docker_name=230755935769.dkr.ecr.us-east-1.amazonaws.com/smore-byoc

env: docker_name=230755935769.dkr.ecr.us-east-1.amazonaws.com/smore-byoc


In [4]:
!pygmentize ./byoc/smore/build_and_push.sh

#!/bin/bash
image=$1
if [ "$image" == "" ]
then
    echo "Usage: $0 <image-name>"
    exit 1
fi
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

$(aws ecr get-login --region ${region} --no-include-email)

docker build -t ${image} . 
docker tag ${image} ${fullname}

docker push ${fullname}


In [5]:
!cat ./byoc/smore/dockerfile

FROM ubuntu:18.04


RUN apt-get update && \
    apt-get install -y --no-install-recommends \
        ca-certificates \
        cmake \
        build-essential \
        gcc \
        g++ \
        git && \
    rm -rf /var/lib/apt/lists/*

RUN git clone https://github.com/cnclabs/smore.git && cd smore && make 

RUN cp -r /smore /app/
ENV PATH="/app:${PATH}"
WORKDIR /app 
COPY train /app/





### Data Preparation 

In [6]:
import sys
sys.path.append('../')

from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.smore_transformer import SmoreDataTransformer
import pickle 

user_item  = pickle.load(open("data/user_item.p", "rb")) 


users = {}
items = {}
train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]
transformer = SmoreDataTransformer(users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(users, items, test_user_item)

In [7]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/smore-movielens'
train_key      = 'net'
train_prefix   = '{}/{}'.format(prefix, 'train')
output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)


import io,boto3
import sagemaker.amazon.common as smac

def writeDatasetToSmoreTXT(X, bucket, prefix, key, Y=None):
    f_name = key+".txt"
    f = open(f_name, 'w')
    for i in range(0, len(X)): 
        raw = X[i]
        n1 = raw[0][0]
        n2 = raw[1][0]
        w = Y[i]
        f.write("{} {} {}\n".format(n1, n2, w))
    
    s3_client = boto3.client('s3')
    object_name = '{}/{}'.format(prefix, f_name)
    response = s3_client.upload_file(f_name, bucket, object_name)
 
    return 's3://{}/{}'.format(bucket,object_name)
    
train_data = writeDatasetToSmoreTXT(X_train, bucket, train_prefix, train_key,  Y_train)    
  
print('Output: {}'.format(output_prefix))

Output: s3://recommendation-demo-yianc/sagemaker/smore-movielens/output


In [8]:
!cat net.txt | head -n 5

140 1554 1.0
140 1588 1.0
140 1601 1.0
140 1476 1.0
140 1463 1.0
cat: write error: Broken pipe


In [9]:
!mkdir ${PWD}/model
!docker run -it -v ${PWD}:/opt/ml/input/data/train/ -v ${PWD}/model:/opt/ml/model smore-byoc train 

mkdir: cannot create directory ‘/home/ec2-user/SageMaker/recommender/algorithms/model’: File exists
mkdir: cannot create directory '/opt/ml/model': File exists
Connections Preview:
	# of connection:	80000
Connections Loading:
	Progress:		100.00 %
	# of vertex:		2367
Build the Alias Method:
	Reconstructing Graph ...
	Building Alias Tables ...
	Finished.
Model Setting:
	dimension:		64
Model:
	[DeepWalk]
Learning Parameters:
	walk_times:		1
	walk_steps:		40
	window_size:		5
	negative_samples:	5
	alpha:			0.025
	workers:		4
Start Training:
	Alpha: 0.025000	Progress: 100.00 %
Save Model:
	Save to </opt/ml/model/rep_dw.txt>


In [10]:
cat ${PWD}/model/rep_dw.txt | head -n 5 

2367 64
140 -0.072039 0.0465892 -0.199141 -0.335169 -0.0614013 0.0421625 -0.0592508 -0.329073 0.320188 0.0878671 -0.102877 0.147127 0.0273161 0.0687555 -0.276137 -0.311322 0.0679498 -0.120628 0.0157252 -0.130048 -0.0641683 -0.0720416 0.0164796 -0.547624 0.159068 0.359149 0.0890028 0.0611993 -0.207284 0.139362 -0.0662043 -0.0819673 -0.129117 0.2181 -0.066151 -0.073682 -0.0373574 0.246498 -0.198885 -0.0594213 -0.0413026 -0.00376554 0.0363313 0.269003 -0.0280695 -0.102051 -0.0639101 -0.369932 0.0482646 -0.451708 -0.0996484 -0.13991 0.00758065 0.148217 -0.174237 0.121686 0.0252737 -0.211 -0.119822 0.0584288 -0.0695208 -0.0220653 -0.115582 0.0414145
1554 -0.0795997 0.0433838 -0.192323 -0.311289 -0.059076 0.0497207 -0.0528664 -0.326663 0.316134 0.0956682 -0.0866561 0.143001 0.0268247 0.0682286 -0.264156 -0.306317 0.0685595 -0.12497 0.0160194 -0.120207 -0.0460327 -0.0632964 0.0179489 -0.522151 0.154211 0.341895 0.0963406 0.0641499 -0.20128 0.149819 -0.0596729 -0.0712739 -0.126534 0.214925 -0.

In [11]:
import boto3 
import os 
import sagemaker 
from sagemaker import get_execution_role



region = boto3.Session().region_name
crole = get_execution_role()
container = os.environ['docker_name']

smore = sagemaker.estimator.Estimator(image_name=container,
                                   role=crole, 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())



smore.fit({'train': train_data})


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-08-20 08:53:04 Starting - Starting the training job...
2020-08-20 08:53:05 Starting - Launching requested ML instances.........
2020-08-20 08:54:46 Starting - Preparing the instances for training...
2020-08-20 08:55:35 Downloading - Downloading input data...
2020-08-20 08:55:41 Training - Downloading the training image..mkdir: cannot create directory '/opt/ml/model': File exists
Connections Preview:
#011# of connection:#0110#015#011# of connection:#01110000#015#011# of connection:#01120000#015#011# of connection:#01130000#015#011# of connection:#01140000#015#011# of connection:#01150000#015#011# of connection:#01160000#015#011# of connection:#01170000#015#011# of connection:#01179665
Connections Loading:
#011Progress:#011#0110.00 %#015#011Progress:#011#01112.55 %#015#011Progress:#011#01125.10 %#015#011Progress:#011#01137.66 %#015#011Progress:#011#01150.21 %#015#011Progress:#011#01162.76 %#015#011Progress:#011#01175.31 %#015#011Progress:#011#01187.87 %#015#011Progress:#011#011100.0

In [ ]:
import os 
model_file_name = "model.tar.gz"
model_full_path = smore.output_path +"/"+ smore.latest_training_job.job_name +"/output/"+model_file_name
print ("Model Path: ", model_full_path)

#Download FM model 
os.system("aws s3 cp "+model_full_path+ " .")

#Extract model file for loading to MXNet
os.system("tar xzvf "+model_file_name)



In [ ]:
!ls -al 

In [ ]:
!pip install annoy 

In [ ]:
from annoy import AnnoyIndex
from preprocessing.smore_datareader import SmoreDataReader
#to be modified 
smorereader = SmoreDataReader(transformer.u_idx, transformer.i_idx, "rep_dw.txt")
user_vectors = smorereader.read_user_data()
item_vectors = smorereader.read_item_data()

dim = 64
t = AnnoyIndex(dim, 'euclidean')  # Length of item vector that will be indexed
for k, v in item_vectors.items():
    t.add_item(int(k), v)

t.build(100)
t.save('smore.ann')



In [ ]:
import json

import numpy 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


user_item_df = pd.read_pickle("data/user_item_df.p")
item_df = pd.read_pickle("data/item_df.p")
genres = ['unknown','Action' , 'Adventure', 'Animation', 'Childrens' , 'Comedy' , 'Crime', \
                                        'Documentary', 'Drama' ,'Fantasy' , 'Film-Noir' , 'Horror' , 'Musical', \
                                        'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


user_inv_idx = {} 
item_inv_idx = {} 

for k, v in transformer.u_idx.items():
    user_inv_idx[v] = k 
    
for k, v in transformer.i_idx.items():
    item_inv_idx[v] = k 


def plot_heat_map(df, figsize=(10,7)): 
    df = df.div(df.sum(axis=1), axis=0)     
    plt.subplots(figsize=figsize)
    sns.heatmap(df)


# test_user_idx = 89
test_user_idx = 200
u_id = user_inv_idx[test_user_idx]

tester_df = user_item_df[user_item_df['uid']==int(u_id)]
tester_df['positive'] = tester_df['rating'] >3 

review = tester_df[['positive']+genres].groupby(['positive']).sum()
plot_heat_map(review, figsize=(10,5))

In [ ]:
import urllib.parse

def get_imdb_query(q):
    return 'https://www.imdb.com/find?q={}'.format(urllib.parse.quote(q))

def print_movie_title(df):
    imdb_search_url = ""
    for index, row in df.iterrows():
        print (row['title'], get_imdb_query(row['title']))


test_positive_df = user_item_df[(user_item_df['uid']==int(u_id)) & (user_item_df['rating']>3)][['title']]
print_movie_title(test_positive_df[0:10])




In [ ]:
test_negative_df = user_item_df[(user_item_df['uid']==int(u_id)) & (user_item_df['rating']<3)][['title']]
print_movie_title(test_negative_df[0:10])

In [ ]:
nns = t.get_nns_by_vector(user_vectors[str(test_user_idx)], 100, search_k=-1, include_distances=False) 

for movie_id in nns[:10]:
    print_movie_title(item_df[item_df['iid']==int(movie_id)])

In [ ]:
import numpy as np 

item_df = pd.read_pickle("data/item_df.p")
genres = ['unknown','Action' , 'Adventure', 'Animation', 'Childrens' , 'Comedy' , 'Crime', \
                                        'Documentary', 'Drama' ,'Fantasy' , 'Film-Noir' , 'Horror' , 'Musical', \
                                        'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

item_genres_df = item_df[['iid']+genres]


popularity = user_item_df[user_item_df['rating']>3].groupby('iid').count()['uid'].to_dict() 

    


def get_answers():
    answer = {} 
    for ui in test_user_item: 
        uid = ui[0]
        iid = ui[1]
        rating = ui[2]
        if uid not in answer:  
            answer[uid] = set() 
        if rating > 0: 
            answer[uid].add(iid)
    return answer
                

            
def similarity(genre_df1, genre_df2): 
    sim = 0 
    for i, c in enumerate(genre_df1.columns): 
        if c != 'iid':
            if genre_df1[c].iloc[0] == genre_df2[genres][c].iloc[0]: 
                sim += 1 
    return sim / len(genres) 
           
    
answer = get_answers() 

def get_metrics_related_to_recommendation(user_vectors,u_idx,t,answer):
    all_pop = 0 
    hits = 0 
    topk = 10
    valid_user_nb = 0 
    rcmded = set()
    diversity = [] 
    
    for uid in answer.keys():
        if uid in u_idx.keys() and uid in user_vectors: 
            nns = t.get_nns_by_vector(user_vectors[uid], 100, search_k=-1, include_distances=False) 
            pred = set()
            valid_user_nb += 1 
            all_sim = []
            for i, movieid in enumerate(nns[:topk]): 
                rcmded.add(movieid)
                if movieid in popularity: 
                    all_pop += popularity[movieid] 
                pred.add(str(movieid))
                for j, movieid_other in enumerate(nns[:topk]):
                    if j < i:
                        movie_a_df = item_genres_df[item_genres_df['iid'] == int(movieid)]
                        movie_b_df = item_genres_df[item_genres_df['iid'] == int(movieid_other)]
                        sim = similarity(movie_a_df, movie_b_df)
                        all_sim.append(sim) 
            diversity.append(1/np.mean(np.asarray(all_sim)))             

            hits += len(pred.intersection(answer[uid]))
    print("Novelty:{}".format(1/(all_pop/topk/valid_user_nb)))
    print("Hits@10:{}".format(hits))
    print("Coverage:{}".format(len(rcmded)))
    print("Diversity:{}".format(np.mean(np.asarray(diversity))))
    
get_metrics_related_to_recommendation(user_vectors,transformer.u_idx,t,answer)    

In [ ]:
#Extract model data
import numpy as np 
import pandas as pd 
from preprocessing.smore_datareader import SmoreDataReader


smorereader = SmoreDataReader(transformer.u_idx, transformer.i_idx, "rep_dw.txt")
user_vectors = smorereader.read_user_data()
item_vectors = smorereader.read_item_data()
 



vectors_to_visualize = [] 
genre_labels = [] 
for k, v in item_vectors.items(): 
    iid = k
    for index, item_info in item_df[item_df['iid']==int(iid)].iterrows():     
        for g in genres: 
            if item_info[g] > 0:
                vectors_to_visualize.append(np.asarray(v)) 
                genre_labels.append(g)
                
    


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

vectors_to_visualize = np.asarray(vectors_to_visualize)                
pca = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300)
pca_result = pca.fit_transform(vectors_to_visualize)
d = {} 
d['pca-1'] = pca_result[:,0]
d['pca-2'] = pca_result[:,1]
d['genre'] = genre_labels

df = pd.DataFrame(data=d)

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-1", y="pca-2",
    hue="genre",
    data=df,
    legend="full",
    alpha=0.3
)